## Checking paired TSeries dirs

In [2]:
import os
import glob
import pandas as pd
from datetime import datetime
import lab
from lab.classes import database, ExperimentSet
from lab.misc import dir_start_time
import glob

/home/gergely/anaconda2/lib/python2.7/site-packages/statsmodels/nonparametric/kde.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .linbin import fast_linbin
/home/gergely/anaconda2/lib/python2.7/site-packages/statsmodels/nonparametric/smoothers_lowess.py:11: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._smoothers_lowess import lowess as _lowess
/home/gergely/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/home/gergely/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/home/gergely/anaconda2/lib/python2.7/site-pa

In [3]:
# experiment set
expt_set = ExperimentSet("mossy.sql")

In [4]:
mouseID=("gtm4_1")
mouse = expt_set.grabMouse(mouseID)

In [204]:
mouse

<Mouse: mouseID = gtm4_5,>

In [205]:
# list all of the mice in this experiment set
expt_set.root.getchildren()

/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  


[<Mouse: mouseID = gtm4_1,>,
 <Mouse: mouseID = da002,>,
 <Mouse: mouseID = da_002,>,
 <Mouse: mouseID = gtm4_2,>,
 <Mouse: mouseID = gtm4_3,>,
 <Mouse: mouseID = gtm4_4,>,
 <Mouse: mouseID = gtm4_5,>,
 <Mouse: mouseID = gt2_39,>,
 <Mouse: mouseID = gt2_40,>,
 <Mouse: mouseID = da001,>]

In [111]:
# list all of the experiments for this mouse
len(mouse.getchildren())

/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  


48

In [112]:
# lists all imaging experiments for the mouse IF they're paired
len(mouse.imagingExperiments())

29

In [5]:
# List of the recorded behavior for the mouse. 
allTrials = database.fetchMouseTrials(mouseID)
# making `allTrials` pretty
startTime = []
# stopTime = []
fileName = []
trial_id = []
for trial in allTrials:
    startTime.append(trial.get('start_time'))
    fileName.append(trial.get('behavior_file'))
    trial_id.append(trial.get('trial_id'))
#     stop.append(trial.get('stop'))

dfTrials = pd.DataFrame({'startTime':startTime, 'fileName':fileName, 'trial_id':trial_id})
dfTrials.style

,fileName,startTime,trial_id
0,/data/gergely/BehaviorData/gtm4_1/notneeded/gtm4_1_20181020143653.tdml,2018-10-20 14:36:53,486
1,/data/gergely/BehaviorData/gtm4_1/gtm4_1_20181020143653.tdml,2018-10-20 14:36:53,89
2,/data/gergely/BehaviorData/gtm4_1/notneeded/gtm4_1_20181020153504.tdml,2018-10-20 15:35:04,485
3,/data/gergely/BehaviorData/gtm4_1/gtm4_1_20181020153504.tdml,2018-10-20 15:35:04,82
4,/data/gergely/BehaviorData/gtm4_1/notneeded/gtm4_1_20181021191030.tdml,2018-10-21 19:10:30,484
5,/data/gergely/BehaviorData/gtm4_1/gtm4_1_20181021191030.tdml,2018-10-21 19:10:30,81
6,/data/gergely/BehaviorData/gtm4_1/gtm4_1_20181022112131.tdml,2018-10-22 11:21:31,80
7,/data/gergely/BehaviorData/gtm4_1/gtm4_1_20181022130012.tdml,2018-10-22 13:00:12,83
8,/data/gergely/BehaviorData/gtm4_1/gtm4_1_20181022142849.tdml,2018-10-22 14:28:49,85
9,/data/gergely/BehaviorData/gtm4_1/gtm4_1_20181023094416.tdml,2018-10-23 09:44:16,79


In [203]:
allTrials[0]

{'behavior_file': '/data/gergely/BehaviorData/gtm4_5/gtm4_5_20181020145920.tdml',
 'mouse_name': 'gtm4_5',
 'start_time': datetime.datetime(2018, 10, 20, 14, 59, 20),
 'tSeries_path': None,
 'trial_id': 178L}

Show whether there are paired trials for this mouse

In [6]:
allExperiments = mouse.findall('experiment')
startTime = []
expType = []
tSeriesDir = []
length = []
trialID = []
day = []
condition = []
session = []
# behaviorLength = []
belt = []
for exp in allExperiments:
#     behaviorLenght = exp.get('stopTime') - exp.get('startTime')
    startTime.append(exp.get('startTime'))
    expType.append(exp.get('experimentType'))
    tSeriesDir.append(exp.get('tSeriesDirectory'))
    length.append(exp.get('trial_length'))
    trialID.append(exp.get('trial_id'))
    belt.append(exp.get('belt'))
    day.append(exp.get('day'))
    session.append(exp.get('session'))
    condition.append(exp.get('condition'))
dfExps = pd.DataFrame({'ImagingStartTime':startTime, 'expType':expType, 'TSeriesDir':tSeriesDir, 'expLength':length,
                   'trial_id':trialID, 'belt':belt, 'day':day, 'session':session, 'condition':condition})
dfExps.style

,ImagingStartTime,TSeriesDir,belt,condition,day,expLength,expType,session,trial_id
0,2018-10-20-14h36m53s,,stripe02m,None,None,600,,None,486
1,2018-10-20-14h36m53s,,stripe02m,None,None,600,,None,89
2,2018-10-20-15h35m04s,,stripe02m,None,None,600,,None,485
3,2018-10-20-15h35m04s,,stripe02m,None,None,600,,None,82
4,2018-10-21-19h10m30s,,stripe02m,None,None,600,,None,484
5,2018-10-21-19h10m30s,,stripe02m,None,None,600,,None,81
6,2018-10-22-11h21m31s,/data/gergely/invivo_DATA/mossy/gtm4_1/10_22_rf/TSeries-10222018-rf_a_d0s0-001/,stripe02m,a,0,720,randomForaging,0,80
7,2018-10-22-13h00m12s,/data/gergely/invivo_DATA/mossy/gtm4_1/10_22_rf/TSeries-10222018-rf_a_d0s1-002/,stripe02m,a,0,720,randomForaging,1,83
8,2018-10-22-14h28m49s,/data/gergely/invivo_DATA/mossy/gtm4_1/10_22_rf/TSeries-10222018-rf_a_d0s2-003/,stripe02m,a,0,720,randomForaging,2,85
9,2018-10-23-09h44m16s,,gergo07,A,0,720,,0,79


In [201]:
# this is equivalent with the next cell
mergedDf = pd.merge(dfTrials, dfExps, on=dfTrials['trial_id'])
mergedDf.head()

,fileName,startTime,trial_id_x,ImagingStartTime,TSeriesDir,belt,condition,day,expLength,expType,session,trial_id_y
0,/data/gergely/BehaviorData/gtm4_5/gtm4_5_20181...,2018-10-20 14:59:20,178,2018-10-20-14h59m20s,,stripe02m,None,None,600,,None,178
1,/data/gergely/BehaviorData/gtm4_5/gtm4_5_20181...,2018-10-20 15:57:05,175,2018-10-20-15h57m05s,,stripe02m,None,None,600,,None,175
2,/data/gergely/BehaviorData/gtm4_5/gtm4_5_20181...,2018-10-21 19:33:11,167,2018-10-21-19h33m11s,,stripe02m,None,None,600,,None,167
3,/data/gergely/BehaviorData/gtm4_5/gtm4_5_20181...,2018-10-22 12:42:08,166,2018-10-22-12h42m08s,/data/gergely/invivo_DATA/mossy/gtm4_5/10_22_r...,stripe02m,a,0,720,randomForaging,0,166
4,/data/gergely/BehaviorData/gtm4_5/gtm4_5_20181...,2018-10-22 14:10:52,190,2018-10-22-14h10m52s,,stripe02m,a,0,720,randomForaging,1,190


### dumping the dataframe in csv

In [202]:
fileName = ('/data/gergely/workingData/mossy/expList_gtm4_5.csv')
mergedDf2 = dfTrials.merge(dfExps, on=dfTrials['trial_id'])
mergedDf2.drop(labels=['trial_id_y'], axis=1, inplace=True)
mergedDf2.to_csv(fileName)

In [138]:
allExperiments[6]

< Experiment: gtm4_1, trial_id = 80, session = 0, lap_tolerance = 0.015, experimentType = randomForaging, laps = 7, stopTime = 2018-10-22-11h33m31s, uniqueLocationKey = , belt = stripe02m, tSeriesDirectory = /data/gergely/invivo_DATA/mossy/gtm4_1/10_22_rf/TSeries-10222018-rf_a_d0s0-001/, track_length = 2000, lap_reset_tag = 7F001B55AD9C, position_scale = -2.07, startTime = 2018-10-22-11h21m31s, trial_length = 720, sensors = [{"report_pin": 20, "action": "create", "type": "lickport", "pin": 2}, {"action": "create", "type": "rfid_reader", "pin": 22}], day = 0, condition = a, contexts = [{"max_duration": 3, "pin": 5, "sensor": 2, "durations": [80], "display_color": [0, 204, 0], "locations": 3, "operant_rate": 2, "initial_open": 0, "drop_size": 80, "action": "create", "radius": 50, "type": "operant", "id": "reward", "valves": [5]}], uses = [{"tag": "_controllers", "file": "controllers.json"}, {"tag": "_rigSettingsExperiment", "file": "experiments.json"}, {"tag": "_opRF", "file": "experimen

In [95]:
for exp in allExperiments:
    print(exp.get('belt'))

stripe02m
stripe02m
stripe02m
stripe02m
stripe02m
stripe02m
stripe02m
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripeCued
stripe02m
gergo05
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07
gergo07


### Pairing TSeries with behavior manually

In [47]:
#use this to pair
database.pairImagingData('/data/gergely/invivo_DATA/mossy/gtm4_3/10_23_hr1/TSeries-10222018-hr_A_d0s0-001',trial_id=102)

--------------------------------------------------------------

In [13]:
allExperiments[0]

< Experiment: gtm4_1, trial_id = 89, lap_tolerance = 0.015, experimentType = , laps = 9, stopTime = 2018-10-20-14h46m53s, uniqueLocationKey = , belt = stripe02m, tSeriesDirectory = , track_length = 2000, lap_reset_tag = 7F001B55AD9C, position_scale = -2.07, startTime = 2018-10-20-14h36m53s, trial_length = 600, sensors = [{"report_pin": 20, "action": "create", "type": "lickport", "pin": 2}, {"action": "create", "type": "rfid_reader", "pin": 22}], contexts = [{"max_duration": 3, "pin": 5, "sensor": 2, "durations": [80], "display_color": [0, 204, 0], "locations": 10, "operant_rate": 2, "initial_open": 0, "drop_size": 80, "action": "create", "radius": 50, "type": "operant", "id": "reward", "valves": [5]}], uses = [{"tag": "_controllers", "file": "controllers.json"}, {"tag": "_rigSettingsTraining", "file": "experiments.json"}, {"tag": "_runTraining", "file": "experiments.json"}], controllers = {"behavior_controller": {"ip": "127.0.0.1", "address": "127.0.0.1:5015", "serial_port": "\\.\\COM5

### Listing TSeries dierctories

In [21]:
mouseDirectory = '/data/gergely/invivo_DATA/mossy/gtm4_1'

In [24]:
for dir, folders, files in os.walk(mouseDirectory):
    print(glob.glob(os.path.join(dir,'TSeries*')))

[]
['/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003', '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-cp-002', '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-ctrl-001']
['/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-003.env', '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-003.xml', '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-003_Cycle00001_Element00001.h5', '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-003.sima']
['/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/MIP/TSeries-11

In [90]:
prairie_xml = glob.glob(mouseDirectory)
prairie_xml

['/data/gergely/invivo_DATA/mossy/gtm4_1']

In [ ]:
prairie_xml = glob.glob

In [36]:
[x[0] for x in os.walk('/data/gergely/invivo_DATA/mossy/gtm4_1')]

['/data/gergely/invivo_DATA/mossy/gtm4_1',
 '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1',
 '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003',
 '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/MIP',
 '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/References',
 '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-003.sima',
 '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-003.sima/s2p',
 '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-003.sima/s2p/suite2p',
 '/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-0

In [7]:
for dir, folders, files in os.walk('/data/gergely/invivo_DATA/mossy/gtm4_1'):
    print(dir)
        

/data/gergely/invivo_DATA/mossy/gtm4_1
/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1
/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003
/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/MIP
/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/References
/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-003.sima
/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-003.sima/s2p
/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-003.sima/s2p/suite2p
/data/gergely/invivo_DATA/mossy/gtm4_1/11_1_pharmacology_CB1/TSeries-11012018-pharm_CB1-wash-003/TSeries-11012018-pharm_CB1-wash-003.sima/s2p/suite2p/plane0
/data/g

[< Experiment: gtm4_1, trial_id = 89, lap_tolerance = 0.015, experimentType = , laps = 9, stopTime = 2018-10-20-14h46m53s, uniqueLocationKey = , belt = stripe02m, tSeriesDirectory = , track_length = 2000, lap_reset_tag = 7F001B55AD9C, position_scale = -2.07, startTime = 2018-10-20-14h36m53s, trial_length = 600, sensors = [{"report_pin": 20, "action": "create", "type": "lickport", "pin": 2}, {"action": "create", "type": "rfid_reader", "pin": 22}], contexts = [{"max_duration": 3, "pin": 5, "sensor": 2, "durations": [80], "display_color": [0, 204, 0], "locations": 10, "operant_rate": 2, "initial_open": 0, "drop_size": 80, "action": "create", "radius": 50, "type": "operant", "id": "reward", "valves": [5]}], uses = [{"tag": "_controllers", "file": "controllers.json"}, {"tag": "_rigSettingsTraining", "file": "experiments.json"}, {"tag": "_runTraining", "file": "experiments.json"}], controllers = {"behavior_controller": {"ip": "127.0.0.1", "address": "127.0.0.1:5015", "serial_port": "\\.\\COM